# Clean(ish) lm-hackers notebook
The purpose of this notebook is to simplify and streamline the fastai notebook lmS-hackers.ipynb that Jeremy Howard worked through in [A Hackers' Guide to Language Models](https://www.youtube.com/watch?v=jkrNMKz9pWU). There is minimal prose and digressions, except to explain parts of the code I didn't previously understand. Also, the openapi seems to have been updated since the youtube video and when the repo was last updated (28th September 2023).

<u>Contents</u>
1. Notebook Prep
2. Authentication
3. Chat Completion

### 1. Notebook Prep

In [158]:
# imports
import ast, openai, torch, textwrap, inspect, json, os
from tiktoken import encoding_for_model
from openai import OpenAI
from pydantic import create_model
from inspect import Parameter
from fastcore.utils import nested_idx

from transformers import AutoModelForCausalLM, AutoTokenizer

In [159]:
print(openai.__version__)

1.8.0


In [18]:
# textwrap function for readability
def wprint(text, width=80):
    print(textwrap.fill(text, width))

### 2. Authentication

If you have an account with OpenAI you can generate an API key on [platform.openai.com/api-keys](https://platform.openai.com/api-keys). You can then either copy the following line into the .zshrc file associated with your environment (if using mac):

```export OPENAI_API_KEY='YOUR KEY HERE'```

or you can set your key as a variable in the jupyter notebook session, which works fine, but is not safe if sharing your code with anyone else, as they will have access to your API key and could use it to make their own API calls that you end up paying for!

Check whether your API key is set as an environment correctly:
```python
print(os.environ.get("OPENAI_API_KEY"))
```

How to set your API key directly in the jupyter notebook session:
```python
client = OpenAI(api_key='YOUR API KEY HERE')
``````

### 3. Chat completion

In [209]:
# default custom instructions recommende by Jeremy
# I have only changed one thing,
    # I have changed the verbosity "flag" at the start of the prompt
    # from "vv" to "cc" which stands for "concise concise" as opposed
    # to v which I associate with "verbose"

cust_inst0 = ("You are an autoregressive language model that has been fine-tuned with instruction-tuning and RLHF."
        " You carefully provide accurate, factual, thoughtful, nuanced answers, and are brilliant at reasoning."
        " If you think there might not be a correct answer, you say so."
        " Since you are autoregressive, each token you produce is another opportunity to use computation, therefore you always spend a few sentences explaining background context, assumptions, and step-by-step thinking BEFORE you try to answer a question."
        " However: if the request begins with the string 'cc' then ignore the previous sentence and instead make your response as concise as possible, with no introduction or background at the start, no summary at the end, and outputting only code for answers where code is appropriate."
        " Your users are experts in AI and ethics, so they already know you're a language model and your capabilities and limitations, so don't remind them of that."
        " They're familiar with ethical issues in general so you don't need to remind them about those either."
        " Don't be verbose in your answers, but do provide details and examples where it might help the explanation."
        " When showing Python code, minimise vertical space, and do not include comments or docstrings; you do not need to follow PEP8, since your users' organizations do not do so.")

# wprint(custom_instructions)

In [148]:
# show how work-to-int encodings work
enc = encoding_for_model('text-davinci-003')
toks = enc.encode('Some words encoded into t to tokens')
print(toks)

[4366, 2456, 30240, 656, 256, 284, 16326]


In [171]:
# function to take system prompt, user prompt and model and performs an api call
def askgpt(prompt, client=None, system=None, model='gpt-3.5-turbo', **kwargs):
    """
    Sends a prompt to a GPT model via OpenAI API and returns the response.

    Parameters:
    prompt (str): User's input prompt for the GPT model.
    client (OpenAI, optional): OpenAI API client. Defaults to a new instance if None.
    system (str, optional): System-level message included in the request.
    model (str, optional): GPT model identifier (default 'gpt-4').
    **kwargs: Extra arguments for chat.completions.create method.

    Returns:
    dict: Response from the GPT model.

    Example:
    >>> response = askgpt("What is the weather today?", model="gpt-4")
    """
    allowed_models = {'gpt-4-1106-preview',
                      'gpt-4',
                      'gpt-4-0314',
                      'gpt-4-0613',
                      'gpt-3.5-turbo',
                      'gpt-3.5-turbo-0301',
                      'gpt-3.5-turbo-0613',
                      'gpt-3.5-turbo-1106'}
    
    if model not in allowed_models:
        print('model must be one of allowed models:')
        for model in allowed_models:
            print(model)
        return None
    
    if client==None:
        client= OpenAI()
    
    m = []
    if system:
        m.append({'role': 'system', 'content': system})

    m.append({'role': 'user', 'content': prompt})
    return client.chat.completions.create(model=model, messages=m, **kwargs)

In [186]:
# function to calculate cost of api call in dollars
def api_cost(response):
    model = response.model

    price_per_k = {'gpt-4-1106-preview': (0.01, 0.03),
                   'gpt-4': (0.03, 0.06),
                   'gpt-4-0314': (0.03, 0.06),
                   'gpt-4-0613': (0.03, 0.06),
                   'gpt-3.5-turbo': (0.001, 0.002),
                   'gpt-3.5-turbo-0301': (0.001, 0.002),
                   'gpt-3.5-turbo-0613': (0.001, 0.002),
                   'gpt-3.5-turbo-1106': (0.001, 0.002),}
    
    k_input_tokens = response.usage.prompt_tokens/1000
    k_output_tokens = response.usage.completion_tokens/1000

    res = price_per_k[model][0]*k_input_tokens + price_per_k[model][1]*k_output_tokens
    
    return res 

In [190]:
# function to print number of tokens and cost of api call
def print_usage(response):
    cost = api_cost(response)
    print(f'Usage, # prompt tokens = {response.usage.prompt_tokens}')
    print(f'Usage, # completion tokens = {response.usage.completion_tokens}')
    print(f'Usage, cost = ${api_cost(response):.5f}')
    print(f'Usage, cost = ¢{api_cost(response)*100:.5f}')

In [210]:
# testing the model
prompt = "cc Write some python code to communicate with a device using a serial connection over ethernet"
model = 'gpt-4'

response = askgpt(prompt, model=model, system=cust_inst0)

In [211]:
# seeing how much the model cost
output = response.choices[0].message.content
print('Output text:')
print(output)

print('\nUsage:')
print_usage(response)

Output text:
```python
import socket
import serial

def connect_to_device(ip, port):
    s = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
    s.connect((ip, port))
    return s

def send_message(sock, message):
    sock.send(message.encode())

def read_message(sock):
    return sock.recv(1024).decode()

def disconnect_from_device(sock):
    sock.close()

sock = connect_to_device('192.168.0.1', 5000)
send_message(sock, 'Some command')
response = read_message(sock)
disconnect_from_device(sock)
```

Usage:
Usage, # prompt tokens = 291
Usage, # completion tokens = 118
Usage, cost = $0.01581
Usage, cost = ¢1.58100


In [205]:
# call the api with some exception handling
# openai API errors documented here:
    # https://platform.openai.com/docs/guides/error-codes/api-errors

def call_api(prompt, system = None, model='gpt-3.5-turbo', max_retries=5):
    """
    Sends a prompt to the OpenAI API and returns the model's response.

    Handles rate limiting with a maximum number of retries and exponential backoff.

    Parameters:
    prompt (str): The prompt to send to the model.
    model (str): Model identifier (default 'gpt-3.5-turbo').
    max_retries (int): Maximum number of retries (default 5).

    Returns:
    dict: The response from the API, or None if an error occurs.
    """

    m = []
    if system:
        m.append({'role': 'system', 'content': system})
    
    m.append({'role': 'user', 'content': prompt})
    
    client = OpenAI()
    retry_count = 0

    while retry_count < max_retries:
        try:
            out = client.chat.completions.create(model=model, messages=m)
            print(f'Retries required: {retry_count}')
            return out
        
        except openai.RateLimitError as e:
            wait_time = int(e.headers.get('retry-after', 5))
            print(f'Rate limit exceeded, waiting for {wait_time} seconds...')
            time.sleep(wait_time)
            retry_count += 1

        except openai.APIError as e:
            print(f'An error occured: {e}')
            return None
        
    print('Maximum retry limit reached')
    return None

out = call_api('cc tell me about the Belgian TV show Undercover',
               system=cust_inst0,
               model='gpt-3.5-turbo')

Retries required: 0


In [206]:
wprint(out.choices[0].message.content)
print_usage(out)

"Undercover" is a Belgian TV show that aired in 2019. It is a crime drama series
that revolves around the story of undercover agents investigating a drug
operation led by a wealthy and elusive criminal. The show gained international
popularity and critical acclaim for its compelling storytelling, well-developed
characters, and immersive atmosphere. It has been praised for its authentic
depiction of the criminal underworld and its exploration of ethical dilemmas
faced by law enforcement agents. The series showcases the challenges and risks
involved in undercover operations while delving into the personal lives of the
characters involved.   The show has received positive reviews for its writing,
acting, and production values, and it has been praised for its ability to
maintain suspense and tension throughout the series. "Undercover" has been well-
received by audiences worldwide and has been highly regarded for its realistic
portrayal of law enforcement, criminal activities, and the mora

### 4. Code Interpreter